<a href="https://colab.research.google.com/github/matinmazid/York_ML_certificate/blob/master/assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load all the dependant libraries

# Scenario 0 No context provided to model

In [2]:
!pip --quiet install langchain langchain_community pypdf langchain_core langchain-text-splitters langchain-chroma
!pip --quiet install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.6 MB/s eta 0:00:0

In [4]:
import os
os.environ['OPENAI_API_KEY']=""

Create the lang chain

In [5]:
from langchain.chat_models import ChatOpenAI
import langchain_core
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
model = ChatOpenAI(temperature=0, model_name = "gpt-3.5-turbo")
output_parser = StrOutputParser()
chatPrompt=ChatPromptTemplate.from_messages(
    [("system","you are a financial specialist with experience in Aritifical Intelligence"),
     ("user","what is XBRL and its relationship to AICPA")]
    )

In [6]:
chain= chatPrompt | model | output_parser

In [7]:
chain.invoke({})


'XBRL stands for eXtensible Business Reporting Language, which is a standardized format for the electronic communication of business and financial data. It allows companies to easily and accurately transmit financial information in a machine-readable format. XBRL tags data using a standardized set of codes, making it easier for investors, analysts, regulators, and other stakeholders to access and analyze financial information.\n\nThe American Institute of Certified Public Accountants (AICPA) is a professional organization for certified public accountants in the United States. The AICPA has been involved in promoting the use of XBRL for financial reporting. The AICPA has developed guidance and resources to help accountants and companies implement XBRL effectively in their financial reporting processes.\n\nIn summary, the AICPA has been supportive of the adoption of XBRL as a tool for improving the accuracy and efficiency of financial reporting, and has provided guidance to its members o

# Scenario 1 Load pdf document loader saving to Chromadb

In [8]:
!pip install --upgrade --quiet pypdf
!pip install --upgrade --quiet langchain_openai
!pip install --quiet tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.2 MB/s eta 0:00:00


https://www.stern.nyu.edu/sites/default/files/assets/documents/ChenChoDouLev2021WP.pdf

https://python.langchain.com/v0.2/docs/how_to/document_loader_pdf/

In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import tiktoken
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

loader = PyPDFLoader("ChenChoDouLev2021WP.pdf")
docs = loader.load()


# split and chuck the input data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
# text splitter knows how to extract the text of a sourced doc
all_splits = text_splitter.split_documents(docs)

# all_texts = [cleanStr(doc.page_content) for doc in all_splits]
# all_texts


vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("what is XBRL and its relationship to AICPA")




/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'XBRL is eXtensible Business Reporting Language, a format required by the SEC for public companies to submit financial reports. The AICPA is not directly mentioned in the provided context in relation to XBRL.'

# Scenario 2 Pine cone database and webcrawler
https://python.langchain.com/docs/integrations/document_loaders/recursive_url/
https://www150.statcan.gc.ca/n1/daily-quotidien/250207/dq250207a-eng.htm


In [11]:
!pip install -qU langchain-community beautifulsoup4 lxml

In [12]:
import re
from bs4 import BeautifulSoup
# import html as htmlTool


def bs4_extractor(html: str) -> str:
    soup = BeautifulSoup(html, "lxml")
    someText=re.sub(r"\n\n+", "\n\n", soup.text).strip()
    # someText=htmlTool.unescape(someText)
    # the test page had non-breaking space as well as other non acii characters
    # we get rid of them
    return someText.encode('latin-1', 'ignore').decode('ascii',"ignore")


In [13]:
from langchain_community.document_loaders import RecursiveUrlLoader

loader=RecursiveUrlLoader(url="https://www150.statcan.gc.ca/n1/daily-quotidien/250207/dq250207a-eng.htm",
                           use_async=False
                          ,extractor=bs4_extractor)

docs_html = loader.load()

# text splitter knows how to extract the text of a sourced doc
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
# text splitter knows how to extract the text of a sourced doc
all_splits = text_splitter.split_documents(docs_html)


In [14]:
all_splits[3]

Document(metadata={'source': 'https://www150.statcan.gc.ca/n1/daily-quotidien/250207/dq250207a-eng.htm', 'content_type': 'text/html; charset=UTF-8', 'title': 'The Daily\xa0—\xa0Labour Force Survey, January 2025', 'description': 'Employment increased by 76,000 (+0.4%) in January and the employment rate rose 0.1 percentage points to 61.1%. The unemployment rate declined 0.1 percentage points to 6.6%.', 'language': 'en'}, page_content='527,000\nJanuary2025\n\n-0.1%\n(monthly change)\n\n Source(s): Table 14-10-0287-01.\r\n\n\r\nUnemployment rate  N.S.\r\n\n5.9%\nJanuary2025\n\n-0.4pts\n(monthly change)\n\n Source(s): Table 14-10-0287-01.\r\n\nTab1\nTab2\n\n\r\nEmployment  N.B.\r\n\n403,000\nJanuary2025\n\n0.7%\n(monthly change)\n\n Source(s): Table 14-10-0287-01.\r\n\n\r\nUnemployment rate  N.B.\r\n\n6.4%\nJanuary2025\n\n-1.3pts\n(monthly change)\n\n Source(s): Table 14-10-0287-01.\r\n\nTab1\nTab2\n\n\r\nEmployment  Que.\r\n\n4,636,000\nJanuary2025\n\n0.3%\n(monthly change)')

Pinecone requires a API key.
https://docs.pinecone.io/guides/get-started/quickstart

In [15]:
!pip install --quiet protobuf==3.20.*
!pip install  --quiet "pinecone[grpc]"
!pip install --upgrade --quiet  lark
!pip install --upgrade --quiet pinecone-notebooks pinecone-client
!pip install --upgrade --quiet langchain_pinecone


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.30.0 requires protobuf<6.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.4 MB/s eta 0:00:00

In [18]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time

pc = Pinecone(api_key="")
os.environ['PINECONE_API_KEY']=""

In [19]:
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

index_name = "york-1020-assignment-2"
# create new index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )


pinecone_vector_store= PineconeVectorStore.from_documents(
    documents=all_splits,
    embedding=OpenAIEmbeddings(),
    index_name=index_name,
    text_key="page_content"
)

pinecone_retriever = pinecone_vector_store.as_retriever()
#
pinecone_rag_chain = (
    {"context": pinecone_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [20]:
pinecone_rag_chain.invoke("how much has employment increased")

'Employment increased by 76,000 in January, with the employment rate rising to 61.1%. The unemployment rate declined to 6.6%. Employment gains were seen across various demographics and industries, with manufacturing and professional services leading the way.'

# Scenario 3 drop box as cloud storage and weaviate

In [51]:
!pip install --quiet dropbox
!pip install --quiet "unstructured[pdf]"
!pip install -U weaviate-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.1/350.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.7 MB/s eta 0:00:00


https://python.langchain.com/docs/integrations/document_loaders/dropbox/
https://python.langchain.com/docs/integrations/vectorstores/weaviate/#connecting-to-weaviate
https://weaviate.io/developers/weaviate/quickstart/local


In [49]:
from langchain_community.document_loaders import DropboxLoader
# Generate access token: https://www.dropbox.com/developers/apps/create.
dropbox_access_token = ""
# Dropbox root folder
dropbox_folder_path = ""
loader = DropboxLoader(
    dropbox_access_token=dropbox_access_token,
    dropbox_folder_path=dropbox_folder_path,
    recursive=True,
)

In [ ]:
documents = loader.load()
